In [2]:
import pandas as pd
from clickhouse_connect import get_client

In [90]:
def create_date_dimension(start_date, end_date):
    dates = pd.date_range(start=start_date, end=end_date, freq='D')
    df = pd.DataFrame(dates, columns=['Date'])
    df['DateKey'] = df['Date'].dt.strftime('%Y%m%d')
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['DayOfMonth'] = df['Date'].dt.day
    df['DayOfWeek'] = df['Date'].dt.dayofweek + 1  # Понедельник = 1, Воскресенье = 7
    df['IsWeekend'] = df['DayOfWeek'].apply(lambda x: 1 if x >= 6 else 0)
    
    # Пример праздников
    holidays = ['2023-12-25', '2023-01-01', '2023-07-04']
    df['IsHoliday'] = df['Date'].dt.strftime('%Y-%m-%d').isin(holidays).astype(int)

    return df


def create_time():
    hours = range(24)  # 24 часа
    minutes = range(60)  # 60 минут

    # Создаем DataFrame с часами и минутами
    time_data = [(hour, minute) for hour in hours for minute in minutes]
    time_df = pd.DataFrame(time_data, columns=['Hour', 'Minute'])

    time_df['Time'] = time_df['Hour'].apply(lambda x: f'{x:02}') + ':' + time_df['Minute'].apply(lambda x: f'{x:02}')

    time_df['TimeKey'] = time_df.index

    time_df = time_df[['TimeKey', 'Hour', 'Minute', 'Time']]

    return time_df


In [77]:
# здесь генерируем и записываем таблицу с датами
date_dimension = create_date_dimension('2023-01-01', '2023-01-01')


client = get_client(
    host='10.1.11.65',
    username='default',  
    password='pegas_warehouse2025',  
    database='loaders_olap' 
)


client.insert_df('calendar', date_dimension)

In [97]:
# здесь генерируем и записываем таблицу со временем
df_time = create_time()


client = get_client(
    host='10.1.11.65',
    username='default',  
    password='pegas_warehouse2025',  
    database='loaders_olap' 
)


client.insert_df('calendar_minute_grain', df_time)